Notebook 3: Naive Bayes
==========================

## Goals for learning
In this assignment, we will:

1) Perform a deep-dive into the Naive Bayes classification algorithm
2) Practice working with the array-oriented programming paradigm
3) Gain experience working with off-the-shelf machine learning and data analysis libraries

## Instructions
* Read through the notebook.
* Answer any plain text questions (replace cell content, "YOUR RESPONSE HERE", with your response).
* Insert your code within the code blocks marked with the comments "# START your code here" and "# STOP your code here".
* Do not use loops, iteration, or recursion in any of the code cells.
* Do not use any "Generative AI" tools or assistants in the creation of your solutions.
* Do not import ot use any libraries other than those already imported outside the "# START your code here" and "# STOP your code here" blocks.
* Run all cells to make sure your code works and you see reasonable results.

## Submission details
* Due: Monday 9/22, 11:59 PM
* [Submission instructions](https://www.cs.oswego.edu/~agraci2/csc461/submission_instructions.html)

## Notebook premise
You have recently been hired as an entry-level ML engineer at a startup. Congratulations!
Your startup has been hired by a large healthcare company to implement a diabetes classifier for them.

You remember from school that **Naive Bayes** classifiers show competitive performance against other types of classifiers, and are relatively easy to implement. You suggest to your team that you implement a Naive Bayes model, and everyone agrees.

Once you get started, you realize that perhaps your professor in college glossed over a few details, and implementing the Naive Bayes won't be quite so simple in practice.

## Naive Bayes Implementation

### Loading the data set
First, we want to load the [Pima Indians Diabetes Database](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database).

Note: This is the same dataset that we used for Notebook #1.
If you need to download the dataset again, please see the "Dataset: Pima Indians Diabetes Database" submodule on Brightspace, 
or download it from [Kaggle](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database).

In [2]:
# Import NumPy and Pandas for loading and manipulating data
import pandas
import numpy

# START your code here
# DATASET_ROOT_DIR='/home/agraci2/data/' # Please edit with your dataset path

dataframe = pandas.read_csv('diabetes.csv')

# STOP your code here

# dataframe = pandas.read_csv(DATASET_ROOT_DIR + 'pima/diabetes.csv')

### Partitioning our data into training and evaluation sets
Next, we partition the available data into a training set and an evaluation set, using an 80/20 split.

In [3]:
# We want to use an 80/20 split between our training and test sets
TRAINING_PERCENTAGE=80

# First, we will use the NumPy library to randomly select entries for either the training set or evaluation set
np_random = numpy.random.RandomState(seed=12345)
rand_unifs = np_random.uniform(0,1,size=dataframe.shape[0]) # A collection of random numbers [0,1) corresponding to each entry in our dataframe
division_threshold = numpy.percentile(rand_unifs, TRAINING_PERCENTAGE) # A threshold that the random numbers above can be checked against to see if they fall into the 80% or 20%

# The training set will use the first 80% of entries
train_indicator = rand_unifs < division_threshold # A collection of True/False indicators corresponding to each entry in our dataframe
train_dataframe = dataframe[train_indicator].reset_index(drop=True) # Filter our dataframe based on the training indicators above

# The test set will use the remaining 20% of entries
eval_indicator = rand_unifs >= division_threshold # A collection of True/False indicators corresponding to each entry in our dataframe (inverse of train_indicator)
eval_dataframe = dataframe[eval_indicator].reset_index(drop=True) # Filter our dataframe based on the evaluation indicators above

# Show how many entries (rows) are in our training vs evaluation dataframes:
print(f'Number of entries in the training set: {len(train_dataframe)}')
print(f'Number of entries in the evaluation set: {len(eval_dataframe)}')

Number of entries in the training set: 614
Number of entries in the evaluation set: 154


### Separating features from labels
Because our classification model requires **labeled data**, we will separate out the label column ('Outcome') from the non-label features. This transformation will need to be performed on both the training and the evaluation sets.

In [5]:
# eval_dataframe.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,3,78,50,32,88,31.0,0.248,26,1
2,10,168,74,0,0,38.0,0.537,34,1
3,0,118,84,47,230,45.8,0.551,31,1
4,7,107,74,0,0,29.6,0.254,31,1


In [4]:
# START your code here
LABEL_COLUMN = 'Outcome'
# STOP your code here

train_features = train_dataframe.loc[:, train_dataframe.columns != LABEL_COLUMN].values # Take all columns except for label
train_labels = train_dataframe[LABEL_COLUMN].values # Take only the label column

eval_features = eval_dataframe.loc[:, eval_dataframe.columns != LABEL_COLUMN].values # Take all columns except for label
eval_labels = eval_dataframe[LABEL_COLUMN].values # Take only the label column

In [6]:
print("Training set (num_rows, num_columns):")
print("- Original training data shape: {}".format(train_dataframe.shape))
print("- New training features shape: {}".format(train_features.shape))
print("- New training labels shape: {}".format(train_labels.shape))
# Note: the absence of a number indicates an implied "1"

print("Evaluation set (num_rows, num_columns):")
print("- Original evaluation data shape: {}".format(eval_dataframe.shape))
print("- New evaluation features shape: {}".format(eval_features.shape))
print("- New evaluation labels shape: {}".format(eval_labels.shape)) 
# Note: the absence of a number indicates an implied "1"

Training set (num_rows, num_columns):
- Original training data shape: (614, 9)
- New training features shape: (614, 8)
- New training labels shape: (614,)
Evaluation set (num_rows, num_columns):
- Original evaluation data shape: (154, 9)
- New evaluation features shape: (154, 8)
- New evaluation labels shape: (154,)


### The Naive Bayes Equation
You review your notes from your ML class, and find the simplified equation for determining the predicted labed $y_p$. It returns the label $y$ for which the product of the **conditional probabilities** for each feature, $P(x^i|y)$, and the **prior probability** of the class, $P(y)$, is the highest.

$$y_p = \textsf{argmax}_{y \in Y}{\left[\prod_i^N{P(x^i|y)}\right]P(y)}$$

### Calculating the prior probability
You decide that calculating the **prior probabilities**, $P(y)$, for each class is a good first step.

<u>Implementation Hints</u>

* For instantiating a NumPy N-Dimensional Array, the [zeros](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html) function may be helpful.
* I found the NumPy [count_nonzero](https://numpy.org/doc/stable/reference/generated/numpy.count_nonzero.html) and [logical_not](https://numpy.org/doc/stable/reference/generated/numpy.logical_not.html) functions to be helpful in my implementation.

In [7]:
# testy testerr

num_data_points = train_labels.shape[0]

label0 = numpy.count_nonzero(train_labels==0)/num_data_points
label1 = numpy.count_nonzero(train_labels)/num_data_points
print(label0)
prior_probabilities = numpy.array([[label0],[label1]])
print(prior_probabilities.shape)

0.6596091205211726
(2, 1)


In [10]:
# This function takes the labels for the training set and determines 
# the prior probability for each class label
def calculate_prior_probabilities(training_labels):
    # This is a binary classifier, so there will only be 2 possible label values ('0' and '1')
    num_classes = 2
    
    # There should only be one prior probability per label
    prior_per_label = 1
    
    # There should be one label per data point in our training set
    num_data_points = training_labels.shape[0]
    
    # TODO: Create a 2x1 array, where the first value is the prior probability
    # for label '0' and the second value is the prior probability for label '1'
    prior_probabilities = None
    
    # START your code 
    label0 = numpy.count_nonzero(training_labels==0)/num_data_points
    label1 = numpy.count_nonzero(training_labels)/num_data_points
    prior_probabilities = numpy.array([[label0],[label1]])
    # STOP your code here
    
    # Use this checkpoint to ensure that your results are in the correct format
    assert prior_probabilities.shape == (num_classes, prior_per_label)
    
    return prior_probabilities

In [11]:
# Checkpoint for prior probability calculations
import math
priors = calculate_prior_probabilities(train_labels)

assert priors.shape == (2, 1)
checked_decimals=5
numpy.testing.assert_almost_equal(priors[0], 0.65960912, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(priors[1], 0.34039088, decimal=checked_decimals, verbose=True)
print("Passed checkpoint!")

Passed checkpoint!


### Calculating conditional probabilities
Now that you are on a roll, you decide to tackle the challenge of computing the **conditional probabilities** for each feature, given each class, $P(x^i|y)$.

However, once you stop to think about it, the simple **frequency distribution** that your professor used in class (for categorical data types) won't work for your data!

In [12]:
# Check the data types in each column
dataframe.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

A quick search from your favorite search engine tells you that using a [**Normal Distribution**](https://en.wikipedia.org/wiki/Normal_distribution), or Gaussian Distribution, is a reasonable intitial choice for calculating distributions with continuous numbers.
$$f(x) = {1 \over \sigma \sqrt{2 \pi}}e^{- {1 \over 2} {\left({x-\mu \over \sigma}\right)^2}}$$

Calculating the **conditional probabilities** for each (feature, class) pair with a Gaussian distribution involves calculating the arithmetic **mean** ($\mu$) and **standard deviation** ($\sigma$) for each (feature, class) pair. Although the equation looks complicated, you think that you can break down the problem a bit. You decide to start implementing just the mean and standard deviation functions.

Equation for Arithmetic Mean:
$$\mu={1 \over M}\sum_i^M{x_i}$$

Equation for Standard Deviation:
$$\sigma=\sqrt{{\sum{(x^i - \mu)^2}} \over M}$$

<u>Implementation Hints</u>

* For instantiating a NumPy N-Dimensional Array, the [zeros](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html) function may be helpful.
* The NumPy [sum](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.sum.html) function was helpful for my implementation.
* Review how arithmetic operators work with NumPy N-Dimensional arrays

In [82]:
# dataframe.columns.tolist()

In [74]:
# print(train_features[:,0],'\n------------------------------------------------\n',train_dataframe['Pregnancies'])

In [79]:
# print(numpy.mean(train_features[:,0]))

In [80]:
# numpy.shape(numpy.zeros((8,2)))

In [93]:
# label0 = (train_labels==0)
# label1 = (train_labels==1)

# ftr0 = train_features[label0]
# # print(label0,'\n',label1)
# print(ftr0.shape)
# print(train_labels.shape)

(405, 8)
(614,)


In [94]:
def calculate_means(training_features, training_labels):
    # This is a binary classifier, so there will only be 2 possible label values ('0' and '1')
    num_classes = 2
    
    # There should be one known label per data point in our training set
    num_data_points = training_labels.shape[0]
    assert num_data_points == training_features.shape[0]
    
    # There should be 8 features for each of our data points
    num_features = training_features.shape[1]
    assert num_features == 8
    
    # TODO: Create a 8x2 array, where there is a mean (mu) of the values for each feature
    # x_i given each class label y_j
    mu = None
    
    # START your code here
    features0 = training_features[training_labels==0]
    features1 = training_features[training_labels==1]

    mu = numpy.array([features0.mean(axis=0), features1.mean(axis=0)]).T
    # STOP your code here
    
    # Use this checkpoint to ensure that your results are in the correct format
    assert mu.shape == (num_features, num_classes)
    return mu

In [95]:
# Checkpoint for mean calculations
import math
mu = calculate_means(train_features, train_labels)
assert mu.shape == (8, 2)
checked_decimals=5
numpy.testing.assert_almost_equal(mu[0][0], 3.48641975, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[0][1], 4.91866029, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[1][0], 109.99753086, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[1][1], 142.30143541, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[2][0], 68.77037037, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[2][1], 70.66028708, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[3][0], 19.51358025, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[3][1], 21.97129187, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[4][0], 66.25679012, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[4][1], 100.55980861, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[5][0], 30.31703704, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[5][1], 35.1492823, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[6][0], 0.42825926, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[6][1], 0.55279904, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[7][0], 31.57283951, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(mu[7][1], 37.39712919, decimal=checked_decimals, verbose=True)
print("Passed checkpoint!")

Passed checkpoint!


In [96]:
def calculate_standard_deviations(training_features, training_labels):
    # This is a binary classifier, so there will only be 2 possible label values ('0' and '1')
    num_classes = 2
    
    # There should be one known label per data point in our training set
    num_data_points = training_labels.shape[0]
    assert num_data_points == training_features.shape[0]
    
    # There should be 8 features for each of our data points
    num_features = training_features.shape[1]
    assert num_features == 8
    
    # TODO: Create a 8x2 array, where there is a mean (mu) of the values for each feature
    # x_i given each class label y_j
    sigma = None
    
    # START your code here
    features0 = training_features[training_labels==0]
    features1 = training_features[training_labels==1]

    sigma = numpy.array([features0.std(axis=0), features1.std(axis=0)]).T
    # STOP your code here
    
    # Use this checkpoint to ensure that your results are in the correct format
    assert sigma.shape == (num_features, num_classes)
    return sigma

In [97]:
# Checkpoint for standard deviation calculations
sigma = calculate_standard_deviations(train_features, train_labels)
assert sigma.shape == (8, 2)
checked_decimals=5
numpy.testing.assert_almost_equal(sigma[0][0], 3.1155426, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[0][1], 3.75417931, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[1][0], 25.96811899, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[1][1], 32.50910874, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[2][0], 18.07540068, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[2][1], 21.69568568, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[3][0], 15.02320635, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[3][1], 17.21685884, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[4][0], 95.63339586, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[4][1], 139.24364214, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[5][0], 7.50030986, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[5][1], 6.6625219, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[6][0], 0.29438217, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[6][1], 0.37201494, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[7][0], 11.67577435, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(sigma[7][1], 11.01543899, decimal=checked_decimals, verbose=True)
print("Passed checkpoint!")

Passed checkpoint!


Now that you have the building blocks ($\mu$ and $\sigma$) for calculating the Gaussian distributions, you decided to plug it into the distribution equation and try it out on the evaluation data.

In [98]:
# This calculates the gaussian distribution for a given class
def calculate_gaussian_dist_y(X, mu, sigma, y):
    mu_y = mu[:,y]
    sigma_y = sigma[:,y]
    return 1.0 / (numpy.sqrt(2.0 * numpy.pi) * sigma_y) * numpy.exp(-numpy.power((X - mu_y) / sigma_y, 2.0) / 2)

In [99]:
gaussian_dist = calculate_gaussian_dist_y(eval_features, mu, sigma, 0)

# Display probabilities for the first evaluation data sample
print("Probabilities for each feature x_i: {}".format(gaussian_dist[0]))

# Display the product of probabilities for the first evaluation data sample
print("Product of probabilities for all features in X: {}".format(numpy.prod(gaussian_dist[0])))

Probabilities for each feature x_i: [0.09247705 0.00526529 0.0217215  0.01561001 0.00328148 0.0483312
 1.0790168  0.00983435]
Product of probabilities for all features in X: 2.7785647377592095e-13


### The log trick
You notice something interesting: some of the probability values you are getting are very small (close to zero). You realize that if you multiply many of these very small numbers together as indicated by your equation, they might not be representable as floating point numbers, and might be represented as zero!

$$y_p = \textsf{argmax}_{y \in Y}{\left[\prod_i^N{P(x^i|y)}\right]P(y)}$$

If that happens, your calculations for each class will be zero, and you will have no way to compare the relative probability of an item belonging to one class or another.

What can you do about this?

Since you are a bit stuck at this point, you reach out to your new work friend and describe the problem. She lends you one of her favorite ML books, and hopes that there might be something in there that can help:
> [Forsyth, David. (2019). Applied Machine Learning. 10.1007/978-3-030-18114-7.](https://link.springer.com/book/10.1007/978-3-030-18114-7)

In this book, the author suggests an approach to dealing with the problem of small numbers.
By leveraging the [monotonic](https://en.wikipedia.org/wiki/Monotonic_function) property of [logarithms](https://en.wikipedia.org/wiki/Logarithm) (namely that when $a > b$ then $log(a) > log(b)$ also), you can do your calculations with much larger numbers, while ensuring that you are finding the class with the greatest probability.

$$\textsf{argmax}_{y \in Y}{\left[\prod_i^N{P(x^i|y)}\right]P(y)} = \textsf{argmax}_{y \in Y}{log \left(\left[\prod_i^N{P(x^i|y)}\right]P(y)]\right)}$$

Combining this property with the fact that the log of a product is the sum of the logs (i.e., $log_a(xy) = log_a(x) + log_a(y)$), leads to an alternative representation of our Naive Bayes equation that adds logarithms of probabilities, rather than multiplying the probabilities:

$$y_p = \textsf{argmax}_{y \in Y}{\left[\sum_i^N{log(P(x^i|y))}\right]+log(P(y))}$$

You decide that this approach looks promising, and make a mental note to thank your new work friend later.

<u>Implementation Hint</u>

* The [NumPy "log" function](https://numpy.org/doc/stable/reference/generated/numpy.log.html) can be helpful

In [114]:
# features0 = train_labels==0
# features1 = train_labels==1

# prior_probabilities = calculate_prior_probabilities(train_labels)
# log_prior_probabilities = numpy.array(numpy.log(prior_probabilities))

# print(log_prior_probabilities.shape)

In [112]:
# Calculate the log of the prior probabilities for each class
def calculate_log_prior_probabilities(training_labels):
    # You can use the function you already wrote to get the prior probabilities
    prior_probabilities = calculate_prior_probabilities(training_labels)
    
    # TODO: Create a 2x1 array, where the first value is the log of the prior probability
    # for label '0' and the second value is the log of the prior probability for label '1'
    log_prior_probabilities = None
    
    # START your code here
    log_prior_probabilities = numpy.array(numpy.log(prior_probabilities))
    # STOP your code here
    
    # Use this checkpoint to ensure that your results are in the correct format
    assert log_prior_probabilities.shape == (2,1)
    
    return log_prior_probabilities

In [113]:
# Checkpoint for log prior probability calculations
import math
log_priors = calculate_log_prior_probabilities(train_labels)
assert log_priors.shape == (2, 1)
checked_decimals=5
numpy.testing.assert_almost_equal(log_priors[0], -0.41610786, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(log_priors[1], -1.07766068, decimal=checked_decimals, verbose=True)
print("Passed checkpoint!")

Passed checkpoint!


You decide to go return your friend's book, and tell her the good news about logarithms.
When she hears the approach you are taking, it reminds her that there may be some existing functions you can repuropse from an old project!

These functions calculate the log of the gaussian distribution when given:

* A feature vector ($X$)
* The means for each feature ($\mu$), given a class $y$
* The standard deviations for each feature ($\sigma$), given a class $y$

In [115]:
# This calculates the log gaussian distribution for a given class
def calculate_log_gaussian_dist_y(X, mu, sigma, y):
    mu_y = mu[:,y]
    sigma_y = sigma[:,y]
    first_half = numpy.log(1 / (sigma_y * math.sqrt(2 * math.pi)))
    second_half = 0.5 * pow(((X - mu_y) / sigma_y), 2)
    return first_half - second_half

# This calculates the log gaussian distribution for every class
def calculate_log_gaussian_dist(X, mu, sigma):
    M = X.shape[0]
    N = X.shape[1]
    log_p_x_y_arr = numpy.zeros((M,N,2))
    log_p_x_y_arr[:,:,0] = calculate_log_gaussian_dist_y(X, mu, sigma, 0)
    log_p_x_y_arr[:,:,1] = calculate_log_gaussian_dist_y(X, mu, sigma, 1)
    return log_p_x_y_arr

In [116]:
log_gaussian_dist = calculate_log_gaussian_dist_y(eval_features, mu, sigma, 0)

# Display probabilities for the first evaluation data sample
print("Probabilities for each feature x_i: {}".format(log_gaussian_dist[0]))

# Display the product of probabilities for the first evaluation data sample
print("Sum of probabilities for all features in X: {}".format(numpy.sum(log_gaussian_dist[0])))

Probabilities for each feature x_i: [-2.38079475 -5.2466187  -3.82945292 -4.15984316 -5.71946084 -3.02967803
  0.07605026 -4.62187357]
Sum of probabilities for all features in X: -28.91167169592075


Much better!

### Putting it all together

Now that you have everything you need to implement the new log-based rule for Naive Bayes, all that is left is to put the pieces together.

Remember that this is the equation we are trying to solve:
$$y_p = \textsf{argmax}_{y \in Y}{\left[\sum_i^N{log(P(x^i|y))}\right]+log(P(y))}$$

The function below, "calculate_log_p_x_y" should calculate the following for each of our class labels $y$:

$${\left[\sum_i^N{log(P(x^i|y))}\right]+log(P(y))}$$

In [128]:
def calculate_log_p_x_y(X, mu, sigma, log_prior):
    # This is a binary classifier, so there will only be 2 possible label values ('0' and '1')
    num_classes = 2
    
    # There should be one known label per data point in our training set
    num_data_points = X.shape[0]
    
    # There should be 8 features for each of our data points
    num_features = X.shape[1]
    assert num_features == 8
    
    # Use the functions that your work friend gave you to get log(P(x | y)) for x in X
    # Note: I am naming this variable "log_p_x_y_arr" because it has entries for each of 
    # the 8 features separated out (features are an array). It is your job to add them up!
    log_p_x_y_arr = calculate_log_gaussian_dist(X, mu, sigma)
    assert log_p_x_y_arr.shape == (num_data_points, num_features, num_classes)
    
    # TODO: Create an Mx2 array, where for each data item, you calculate
    # the sum of the logs of the conditional probabilities for each feature 
    # plus the prior probability for the class:
    # log(P(x0 | y)) + log(P(x1 | y)) + ... + log(P(xN | y)) + P(y)
    log_p_x_y = None
    
    # START your code here
    log_p_x_y = log_p_x_y_arr.sum(axis=1) + log_prior.flatten()
    # STOP your code here
    
    assert log_p_x_y.shape == (num_data_points,num_classes)
    return log_p_x_y

In [129]:
# Checkpoint for log_p_x_y calculations
mu = calculate_means(train_features, train_labels)
sigma = calculate_standard_deviations(train_features, train_labels)
log_prior = calculate_log_prior_probabilities(train_labels)
log_p_x_y = calculate_log_p_x_y(train_features, mu, sigma, log_prior)

assert log_p_x_y.shape == (614, 2)
checked_decimals=5
numpy.testing.assert_almost_equal(log_p_x_y[0][0], -26.96647828, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(log_p_x_y[0][1], -31.00418408, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(log_p_x_y[613][0], -26.98605248, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(log_p_x_y[613][1], -30.80571318, decimal=checked_decimals, verbose=True)
print("Passed checkpoint!")

Passed checkpoint!


Finally, you decide to wrap you classification model up as a class so that it is easier for your team and your customers to work with.

In [130]:
class NaiveBayesClassifier():
    def __init__(self, training_features, training_labels):
        
        # The following 3 lines constitute the training step
        # We are creating our model from the cached probability values
        self.log_priors = calculate_log_prior_probabilities(training_labels)
        self.mu = calculate_means(training_features, training_labels)
        self.sigma = calculate_standard_deviations(training_features, training_labels)
        
    def predict(self, features):
        
        # Use the cached probability values along with the observed values of 
        # the input feature vectors to make predictions
        log_p_x_y = calculate_log_p_x_y(features, self.mu, self.sigma, self.log_priors)
        return log_p_x_y.argmax(axis=1)

Now it is time to try your classifier out!
Below, we use your implementation to make predictions on the training and the evaluation data sets, and we calculate accuracy for both cases.

In [131]:
diabetes_classifier = NaiveBayesClassifier(train_features, train_labels)
train_prediction = diabetes_classifier.predict(train_features)
eval_prediction = diabetes_classifier.predict(eval_features)

In [132]:
train_accuracy = (train_prediction==train_labels).mean()
eval_accuracy = (eval_prediction==eval_labels).mean()
print("Training accuracy: {}, Evaluation accuracy: {}".format(train_accuracy, eval_accuracy))

Training accuracy: 0.7671009771986971, Evaluation accuracy: 0.7532467532467533


In [133]:
checked_decimals=7
numpy.testing.assert_almost_equal(train_accuracy, 0.7671009771986971, decimal=checked_decimals, verbose=True)
numpy.testing.assert_almost_equal(eval_accuracy, 0.7532467532467533, decimal=checked_decimals, verbose=True)
print("Passed checkpoint!")

Passed checkpoint!
